# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открытие датасета

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
pd.set_option('display.float_format', '{:.2f}'.format)
m=Mystem()

data = pd.read_csv('/datasets/data.csv') # Входные данные от банка — статистика о платёжеспособности клиентов.
data.head() #осмотр датасета


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


In [2]:
data.info() #

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
(data.isnull().sum()/data.count()).apply('{:.0%}'.format)

children             0%
days_employed       11%
dob_years            0%
education            0%
education_id         0%
family_status        0%
family_status_id     0%
gender               0%
income_type          0%
debt                 0%
total_income        11%
purpose              0%
dtype: object

**Вывод**  в столбцах total_income(ежемесячный доход) и days_employed (общий трудовой стаж в днях), есть пропущенные  значения их 11 %, чтобы это могло значить ? Возможно, что это безработные или пенсионеры, все же проверим.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посмотрим, что за строки содержат пропуски

In [4]:
data[data['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,nan,65,среднее,1,гражданский брак,1,M,пенсионер,0,nan,сыграть свадьбу
26,0,nan,41,среднее,1,женат / замужем,0,M,госслужащий,0,nan,образование
29,0,nan,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,nan,строительство жилой недвижимости
41,0,nan,50,среднее,1,женат / замужем,0,F,госслужащий,0,nan,сделка с подержанным автомобилем
55,0,nan,54,среднее,1,гражданский брак,1,F,пенсионер,1,nan,сыграть свадьбу
65,0,nan,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,nan,операции с коммерческой недвижимостью
67,0,nan,52,высшее,0,женат / замужем,0,F,пенсионер,0,nan,покупка жилья для семьи
72,1,nan,32,высшее,0,женат / замужем,0,M,госслужащий,0,nan,операции с коммерческой недвижимостью
82,2,nan,50,высшее,0,женат / замужем,0,F,сотрудник,0,nan,жилье
83,0,nan,52,среднее,1,женат / замужем,0,M,сотрудник,0,nan,жилье


**Вывод:** Гипотеза о том , что NaN ,безработные или пенсионеры не подтверждается, в поле  income_type  стоят значения  работающих категорий.  Оставим строки с NaN как есть , хоть значения и  отсутсвуют. Исходим из основной цели  "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок" столбцы total_income(ежемесячный доход) и days_employed (общий трудовой стаж в днях) не участвуют в основном выводе. Выкинуть 11% данных может быть опасно. Если что, удалим потом.

In [5]:
data ['median_replace_NaN']= data['total_income'].fillna(data['total_income'].median()).astype(int)#просто медиана
med = data.groupby(['family_status','education_id','gender'])['total_income'].transform('median').astype(int)
data ['median_replace_NaN_group']= data['total_income'].fillna(med).astype(int)# медиана по структуре семейный статус, образование, пол/гендер


data[data ['total_income'].isna()] # смотрим значения NAN и медиан в последних столбцах
                           


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_replace_NaN,median_replace_NaN_group
12,0,nan,65,среднее,1,гражданский брак,1,M,пенсионер,0,nan,сыграть свадьбу,145017,157560
26,0,nan,41,среднее,1,женат / замужем,0,M,госслужащий,0,nan,образование,145017,161549
29,0,nan,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,nan,строительство жилой недвижимости,145017,129126
41,0,nan,50,среднее,1,женат / замужем,0,F,госслужащий,0,nan,сделка с подержанным автомобилем,145017,122687
55,0,nan,54,среднее,1,гражданский брак,1,F,пенсионер,1,nan,сыграть свадьбу,145017,128241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,nan,47,Среднее,1,женат / замужем,0,M,компаньон,0,nan,сделка с автомобилем,145017,161549
21495,1,nan,50,среднее,1,гражданский брак,1,F,сотрудник,0,nan,свадьба,145017,128241
21497,0,nan,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,nan,строительство недвижимости,145017,162822
21502,1,nan,42,среднее,1,женат / замужем,0,F,сотрудник,0,nan,строительство жилой недвижимости,145017,122687


### Обработка дубляжей 

#### Оценка влияния образования на платежную дисциплину

Поле образование скорректируем и приведем к единому стандарту категорий. 

In [6]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [7]:
dict_educat = data.set_index('education_id').to_dict()['education']
dict_educat

{0: 'высшее',
 1: 'среднее',
 2: 'неоконченное высшее',
 3: 'начальное',
 4: 'ученая степень'}

In [8]:
dict(zip(data['education_id'], data['education']))

{0: 'высшее',
 1: 'среднее',
 2: 'неоконченное высшее',
 3: 'начальное',
 4: 'ученая степень'}

Сгруппируем данные. Замениим значение единица и нуль в столбце debt (имел ли задолженность по возврату кредитов) на значения "имел ли задолженность по кредиту", "платил в срок" , так табличка будет читабельнее.

In [9]:
data.loc[data['debt'] == 0, 'debt_1'] = 'Платил в срок' # добаил новый столбец debt_1 для более понятного восприятия
data.loc[data['debt'] == 1, 'debt_1'] = 'Имел задолженность кредиту' # добавил новый столбец debt_1 для более понятного восприятия


In [12]:
rate_bad =data[data['debt'] == 1].groupby(data['education']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['education'])['debt'].agg(['count', 'sum', lambda lovely_format: '{:.2%} '.format(lovely_format.mean())])
rate_good =data.groupby(data['education']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'education', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'education', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'education', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})


,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
education,,,,
высшее,15.97%,24.44%,0.65,5.29%
начальное,1.78%,1.31%,1.36,10.99%
неоконченное высшее,3.91%,3.46%,1.13,9.14%
среднее,78.35%,70.77%,1.11,8.95%


## Оценка влияния социально-демографических характеристик на платежную дисциплину

#### Оценка влияния количества детей клиента на платежную дициплину. 

In [13]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [14]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Вывод** У нас появились  люди, которые рожают по 20 детей и есть кто в минуса пошел. В общем, это все это ошибки, скорее всего  у кого-то рука дрогнула и  нажал нуль или знак минус. Надо скорректировать на руках. 
family_status - все нормально, видимо, разработчики позаботились сделать внесение значений выбором из списка. 

Заменим 20 детей на значение 2 и -1 на 1

In [15]:
data.loc[data['children'] == 20, 'children'] = 2 
data.loc[data['children'] == -1, 'children'] = 1
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**  Необходимо сформировать структуру клиентов, которые имели задолженность по возврату кредитов. 

In [16]:
rate_def= data.groupby(['children'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_def['<lambda_0>']

children
0    7.51% 
1    9.15% 
2    9.48% 
3    8.18% 
4    9.76% 
5    0.00% 
Name: <lambda_0>, dtype: object

In [17]:
rate_bad =data[data['debt'] == 1].groupby(data['children']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['children'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_good =data.groupby(data['children']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'children', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'children', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'children', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})

,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
children,,,,
0,61.06%,65.73%,0.93,7.51%
1,25.56%,22.60%,1.13,9.15%
2,11.60%,9.90%,1.17,9.48%
3,1.55%,1.53%,1.01,8.18%
4,0.23%,0.19%,1.21,9.76%


<div class="alert alert-info">
<b> Вывод:</b> отношение структур показывает, что при наличие иждевенцев, платежная  дисциплина ухудшается. Основная масса клиентов приходится на клиентов не более чем с 2-мя детьми. 
</div>      

#### Оценка влияния семейного положения на платежную дисциплину 
Сгруппируем данные по семейному положению

In [18]:
rate_bad =data[data['debt'] == 1].groupby(data['family_status']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['family_status'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_good =data.groupby(data['family_status']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'family_status', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'family_status', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'family_status', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})

,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
family_status,,,,
Не женат / не замужем,15.74%,13.07%,1.20,9.74%
в разводе,4.88%,5.55%,0.88,7.11%
вдовец / вдова,3.62%,4.46%,0.81,6.56%
гражданский брак,22.29%,19.41%,1.15,9.29%
женат / замужем,53.48%,57.51%,0.93,7.52%


<div class="alert alert-info">
<b>  Вывод:</b> гипотеза, что семейные люди более ответственные подтверждается, но не сказать , что имеет яркую доминанту. Но можно точно сказать, что неженатые/незамужние или состоящие в гражданском браке хуже обслуживают долги.	 
</div>  

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера 2</b>
    
Вот пример того, зачем вообще эти словари нужны. У нас в таблице находятся взаимозаменяемые столбцы. Нам ни к чему дублирующаяся информация. Кроме того, многие алгоритмы плохо работают со строковыми типами данных. Поэтому давай предположим, что мы удалим из таблицы столбец **family_status** после создания словаря.
<div class="alert alert-info">
<h4>Спасибо за ревью.</h4>
</div>    

In [19]:
# КОД РЕВЬЮЕРА

print('Создали словарь:')
family_dict = data[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict)


print('\n\nСгруппированная таблица. Берем по id, другой столбец удалили:')
a = data.groupby('family_status_id')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
display(a)


# Заменяем
print('\n\nЗаменяем численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

Создали словарь:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем




Сгруппированная таблица. Берем по id, другой столбец удалили:


,count,sum,<lambda_0>
family_status_id,,,
0,12380,931,7.52%
1,4177,388,9.29%
2,960,63,6.56%
3,1195,85,7.11%
4,2813,274,9.74%




Заменяем численные значения по ключу словаря:


,family_status_id,count,sum,<lambda_0>
0,женат / замужем,12380,931,7.52%
1,гражданский брак,4177,388,9.29%
2,вдовец / вдова,960,63,6.56%
3,в разводе,1195,85,7.11%
4,Не женат / не замужем,2813,274,9.74%


Рассмотрим оставщиеся поля, не относящиеся к основной  цели исследования, возможно, можно увидеть какие-то концентрации проблем исходя из
- цели кредита
- трудового стажа
- возраста клиента
- образования
- пола клиента
- дохода в месяц

### Категоризация данных

Такие столбцы, как доход в месяц, возраста клиента,трудовой стаж лучше привести к категориям.Для этого создадим функцию, и добавим новые  столбцы с категориями в дата фрейм

In [20]:
data['total_income'].value_counts()#Проверил дубляжи
data['total_income'].min() # посмотрел нет ли отрицательных значений и нулевых
data['total_income'].max() # есть люди с большим доходо 2 265 604, насколько это реально, больше сомнительно
data['total_income'].sort_values(ascending=False).head(20).astype(int)# посмотрел насколько равномерное распраделени масимальных
#зарплат таких людей немного, сильно на вывод не повлияет.  

12412    2265604
19606    2200852
9169     1726276
20809    1715018
17178    1711309
17503    1597613
18368    1551152
18353    1427934
15268    1350245
11071    1286280
2224     1278622
7447     1240165
19813    1223042
15211    1172459
13090    1128836
9871     1103455
10004    1097954
18766    1092608
19338    1091627
1590     1089120
Name: total_income, dtype: int64

Категорируем поле 'доход в месяц'(total_income) тыс руб
от 0 до 50
- 50-100
- 100-150
- 150-300
- 300-500
- 500-1000
- более 1000 тыс руб

In [21]:
#data['total_income'] = data['total_income'].fillna(0)# почищу non a numeric 
#data['total_income'].astype(int)
#data.info

Так как пустые значение NaN заменены на 0, исключу их из группировки по категориям, так как они могут исказить результат.
Создам  функцию по категориям.

In [22]:
def income_group(total_income):

    if 1<=total_income <= 50000:
        return '1-50'
    elif 51000 <= total_income <= 100000:
        return '50-100'
    elif 100001 <= total_income <= 150000:
        return '100-150'
    if 150001 <= total_income <= 300000:
        return '150-300'
    if 300001 <= total_income <= 500000:
        return '300-500'
    if 500001 <= total_income <= 1000000:
        return '500-1000'
    if total_income > 1000000:
        return '>1000'

income_group(150000) #- проверка работы функции
data['income_group'] = data['total_income'].astype(int,errors ='ignore').apply(income_group)

data['income_group'].value_counts()

150-300     7702
100-150     5704
50-100      4055
300-500     1260
1-50         372
500-1000     197
>1000         25
Name: income_group, dtype: int64

<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера </b>
    
Не слишком равнозначное разбиение. Последние 3 группы можно не рассматривать, получается. </div>
<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера </b>
    
Есть еще метод **qcut**, который разбивает выборку на равные группы:</div>

In [23]:
# КОД РЕВЬЮЕРА

pd.qcut(data['median_replace_NaN_group'], q=5)

0        (214404.0, 2265604.0]
1          (98661.6, 128241.0]
2         (128241.0, 161658.4]
3        (214404.0, 2265604.0]
4         (128241.0, 161658.4]
                 ...          
21520    (214404.0, 2265604.0]
21521     (128241.0, 161658.4]
21522     (20666.999, 98661.6]
21523    (214404.0, 2265604.0]
21524     (20666.999, 98661.6]
Name: median_replace_NaN_group, Length: 21525, dtype: category
Categories (5, interval[float64]): [(20666.999, 98661.6] < (98661.6, 128241.0] < (128241.0, 161658.4] < (161658.4, 214404.0] < (214404.0, 2265604.0]]

<div class="alert alert-info">
<b> Комментарий студента</b>
    Классный метод, на обучении не было такой, в методическом материале, вроде, тоже нет. Спасибо.
</div>

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера 2</b>
    
Да, очень удобный :)  </div>

In [24]:
# КОД РЕВЬЮЕРА
pd.qcut(data['median_replace_NaN_group'], 5, ['низкий','ниже среднего','средний','выше среднего', 'высокий'])

0              высокий
1        ниже среднего
2              средний
3              высокий
4              средний
             ...      
21520          высокий
21521          средний
21522           низкий
21523          высокий
21524           низкий
Name: median_replace_NaN_group, Length: 21525, dtype: category
Categories (5, object): [низкий < ниже среднего < средний < выше среднего < высокий]

In [25]:
data['income_qcut'] = pd.qcut(data['total_income'], 5, ['низкий','ниже среднего','средний','выше среднего', 'высокий'])
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_replace_NaN,median_replace_NaN_group,debt_1,income_group,income_qcut
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,253875,253875,Платил в срок,150-300,высокий
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,112080,112080,Платил в срок,100-150,ниже среднего
2,0,-5623.42,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,145885,145885,Платил в срок,100-150,средний
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,267628,267628,Платил в срок,150-300,высокий
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,158616,158616,Платил в срок,150-300,средний
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.32,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.86,операции с жильем,224791,224791,Платил в срок,150-300,высокий
21521,0,343937.40,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.81,сделка с автомобилем,155999,155999,Платил в срок,150-300,средний
21522,1,-2113.35,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.56,недвижимость,89672,89672,Имел задолженность кредиту,50-100,низкий
21523,3,-3112.48,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.05,на покупку своего автомобиля,244093,244093,Имел задолженность кредиту,150-300,высокий


Дополнительно создадим категорию по возрасту, предполагаем , что более молодые люди менее дисциплинированные и их, по идее, должно быть больше в структуре людей которые нарушали платежную дисциплину

#### Оценка влияния уровня дохода на платежную дисциплину

In [26]:
rate_bad =data[data['debt'] == 1].groupby(data['income_group']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['income_group'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_good =data.groupby(data['income_group']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'income_group', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'income_group', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'income_group', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})

,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
income_group,,,,
1-50,1.32%,1.73%,0.76,6.18%
100-150,28.20%,26.50%,1.06,8.61%
150-300,35.61%,35.78%,1.00,8.05%
300-500,5.28%,5.85%,0.90,7.30%
50-100,18.84%,18.84%,1.00,8.09%
500-1000,0.69%,0.92%,0.75,6.09%
>1000,0.11%,0.12%,0.99,8.00%


<div class="alert alert-info">
<b> Комментарий студента</b> Я пока оставил свое распределение. Я считаю ,что людям, наверное, понятнее смотреть цифры, по моим ощущениям, категорировать: средне, выше среднего и так далее не совсем верно , у читателя будем  в голове вопрос, а сколько это выше среднего? и куда ему смотреть? Полагаю, читателям отчета, желательно, дать информацию по проще :). 
</div>    

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера 2</b>
    
Да, разумно.  </div>

<div class="alert alert-info">
<b> Вывод:</b> Структуры практически идентичны. Основная доля продаж более 80%,в диапазоне от 50 до 300 тысяч рублей. На нее же и приходится неплательщики в срок. Людям свойственно завышать доходы, чтобы казаться более платежеспасобными. Но, корректного вывода не сделать из этого пункта не имея цифр по сумме кредита. Надо понимать какие долги обслуживают клиенты с такими доходами. 
</div> 

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера 2</b>
    
Завышать доходы перед банком они не могут. А вывод тут напрашивается очень даже логичный: люди с низким доходом понимают, что не потянут еще и процент за просрочку, поэтому отказывают себе в чем-то, чтобы отложить деньги на выплату кредита. Богатые люди действительно, как правило, более финансово грамотные и кредиты обычно берут на бизнес. А вот средний класс может позволить <div class="alert alert-info">
<h4><b> Комментарии студента 2. </b> </h4><br>
Я, не сосвем согласен с вашим  утверждением, судя по дата сету - это розничное  кредитование. В основном, доходы принимаются со слов заявителей на кредит, допускаю, что небольшая часть может быть со справками о доходах. Я вас уверяю, доходы обычно завышают и  клиенты, чтобы  получить максимальную сумму кредита и менеджеры, которые принимают заявки на кредит для того ,чтобы  получить продажу и  забонусироваться от продажи. Опять-таки оговорюсь, если речь в  дата сете идет о России , и зарплаты сильно большие, у  нас медиана по стране 32 тыс.рублей, тут сильный перекос в сторону высоких доходов, были бы реальные данные, я бы сильно засомневался в их репрезентативности, но тут же учебные. 
</div>     

#### Оценка влияния возраста на платежную дисциплину

In [27]:
data['dob_years'].value_counts()# затесался нуль в  возраст, помечу , что возраст не определен в категории 

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [28]:
def age_group(age):
    if age == 0:
        return 'Возраст не определён'
    if 18 <= age < 25:
        return '18-25'    
    if 25 <= age < 30:
        return '25-30'    
    if 30 <= age < 40:
        return '30-40'
    if 40 <= age < 50:
        return '40-50'   
    if 50 <= age < 60:
        return '50-60'       
    if 60 <= age < 70:
        return '60-70'       
    if age >= 70:
        return 'Пенсионер'
    
#age_group(55)  проверка функции
data['age_group'] = data['dob_years'].apply(age_group)

In [29]:
rate_bad =data[data['debt'] == 1].groupby(data['age_group']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['age_group'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_good =data.groupby(data['age_group']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'age_group', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'age_group', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'age_group', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})

,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
age_group,,,,
18-25,5.05%,4.07%,1.24,10.03%
25-30,14.99%,10.71%,1.40,11.32%
30-40,31.71%,26.36%,1.20,9.73%
40-50,23.21%,24.95%,0.93,7.52%
50-60,17.52%,21.74%,0.81,6.52%
60-70,6.72%,10.90%,0.62,4.99%
Возраст не определён,0.46%,0.47%,0.98,7.92%
Пенсионер,0.34%,0.79%,0.43,3.51%


#### Оценка влияния гендора на платежную дисциплину

In [30]:
data.loc[data['gender'] == 'M', 'gender_1'] = 'Муж'# заменю на более понятные занчения,добавлю новое поле
data.loc[data['gender'] == 'F', 'gender_1'] = 'Жен' # заменю на более понятные занчения,добавлю новое поле
data.loc[data['gender'] == 'XNA', 'gender_1'] = 'Не определен'# заменю на более понятные занчения,добавлю новое поле

In [31]:
rate_bad =data[data['debt'] == 1].groupby(data['gender_1']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['gender_1'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_good =data.groupby(data['gender_1']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'gender_1', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'gender_1', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'gender_1', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})

,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
gender_1,,,,
Жен,57.09%,66.14%,0.86,6.98%
Муж,42.91%,33.86%,1.27,10.25%


### Лемматизация

#### Оценка влияния цели  кредита на платежную дисциплину

In [32]:
data['purpose'].value_counts()
data_uni_goal=data['purpose'].unique()
data_uni_goal

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод:** Да,видимо, разработчки не позаботились сделать список, и предоставили пользователю свободу действия. Необходимо выделить основные цели кредита. Использумем лемматизацию по столбцу цель(porpuse).

In [33]:
%%time
from pymystem3 import Mystem
m = Mystem() 
lemma_list = []

for i in data_uni_goal:
    lemma =  m.lemmatize(i)
    lemma_list.append(lemma)

lemma_list

CPU times: user 2.77 ms, sys: 7.76 ms, total: 10.5 ms
Wall time: 628 ms


[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод:** можно выделить 4 основные цели: 'образование', 'автомобиль', 'недвижимость','свадьба'. 

Сделаем функцию на 4 категории целей и посмотрим, как  там все по уровню нарушений оплаты. 

In [34]:
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'свад' in i:
            return 'свадьбу'
        elif 'образ' in i:
            return  'образование'
        elif 'недвиж'in i:
            return 'недвижимость'
        elif 'автомоб' in i:
            return  'автомобиль'
        elif 'жил'in i: 
            return 'недвижимость'
    return 'не определено'

purpose_change('недвиж') #проверим работу функции



'недвижимость'

In [35]:
%%time
data['purpose_group'] = data['purpose'].apply(purpose_change) # добавляем новый столбец, применяем функцию категоризации       
data['purpose_group'].value_counts()
#data.head(10)

CPU times: user 784 ms, sys: 216 ms, total: 1 s
Wall time: 2.73 s


недвижимость    10840
автомобиль       4315
образование      4022
свадьбу          2348
Name: purpose_group, dtype: int64

In [36]:
rate_bad =data[data['debt'] == 1].groupby(data['purpose_group']).agg({'income_type':'count'})/len(data[data['debt'] == 1])
rate_def= data.groupby(['purpose_group'])['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
rate_good =data.groupby(data['purpose_group']).agg({'income_type':'count'})/len(data)
struct_relation = (rate_bad/rate_good).fillna(0)
merge_table =rate_bad.merge (rate_good, on = 'purpose_group', how='left', suffixes=('_rate_bad', '_rate_good')) # Собираю все в одну табличку
merge_table = merge_table.merge(struct_relation, on = 'purpose_group', how='left') #Собираю все в одну табличку
merge_table = merge_table.merge(rate_def ['<lambda_0>'], on = 'purpose_group', how='left') #Собираю все в одну табличку

merge_table.rename(columns={'income_type_rate_bad': 'Структура клиентов с просрочкой', # Делаю названия  столбцов понятными
              'income_type_rate_good': 'Структура клиентов без просрочки',
              'income_type': 'Структурный свдиг',
              '<lambda_0>':'%% клиентов, имевших просрочку платежа'},inplace=True)

merge_table.style.format({'Структура клиентов с просрочкой': '{:.2%}'.format, # Делаю формат  столбцов понятными
                         'Структура клиентов без просрочки': '{:.2%}'.format,
                        'Структурный свдиг':'{:.2f}'.format})

,Структура клиентов с просрочкой,Структура клиентов без просрочки,Структурный свдиг,"%% клиентов, имевших просрочку платежа"
purpose_group,,,,
автомобиль,23.15%,20.05%,1.15,9.34%
недвижимость,44.92%,50.36%,0.89,7.21%
образование,21.25%,18.69%,1.14,9.20%
свадьбу,10.68%,10.91%,0.98,7.92%


## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Рост количества иждевенцев ухудшает платежную  дисциплину. Уровень просрочки без детей составляет - 7,51%. При наличие детей уровень просрочки составляет, в среднем - 9,3%

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Гипотеза, что семейные более ответсвенныые подтверждается, уровень просрочки по таким клиентам составляет - 7,52%. Также можно точно сказать, что неженатые/незамужние или состоящие в гражданском браке хуже обслуживают долги уровень просрочки по таким клиентам составляет - 9,5%

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Основная доля продаж более 80%,в диапазоне от 50 до 300 тысяч рублей. Уровень просрочки составляет - 8,2%. У клиентов с доходом свыше 300 тыс. руб. Уровень просрочки составляет - 7,2% .

### Как разные цели кредита влияют на его возврат в срок?

Наимение дисциплинированные оказались заемщики с целью кредитования "образование" и "покупка автомобиля, в среднем, уровень просрочки по таким клиентам составил - 9,25%. Наименее рисковая цель кредитования покупка недвижимости уровень просрочки составил - 7.21%

## Шаг 4. Общий вывод

По результатам анализа данных:
- Определено влияние социально-демографических показателей на уровень просрочки.
- Составлен портрет проблемного заемщика
- Даны рекомендации бизнесу.

1. Портетрет проблеменого заемщика.
По индексу соотношения структуры клиентов с нарушением срока погашения кредита к клиентам, которые оплачивали кредит во время можно сделать вывод, что мужчин на 27 % в структуре плохих клиентов больше, чем женщин, молодые люди до 30 лет на 33% хуже обслуживают долги, покупка автомобиля или образования на 14% рискованее, чем другие цели кредитования, неженатые / незамужние также на 23% хуже обсуживают свои долги, чем прочие семейные статусы. Наличие иждивенцев ухудшает оплату кредита во время на 14%, ровным счетом, как отсутствие образования на 11%.
2. Портрет благонадежного заёмщика.
Это женщины состоящие в браке, имеющие высшее образование в возрасте 40 +, скорее всего, не имеющию уже на иждевении детей или их минимум , наиболее предпочтительная цель кредитования - покупка недвижимости. Необходимо проинформировать блок продаж или маркетологов, кого им следует заводить для формирования более качественного кредитного портфеля.
3. Рекомендации.
Кредитному отделу банка также необходимо обратить внимание на портрет благонадежного заёмщика, возможно, к этим клиентам относиться более лояльно. А также обратить внимание на портрет негативного заёмщика. Это молодые(до 30 лет) холостые мужчины, не имеющие образования, желающие себе автомобиль. К таким людям надо присматриваться , а , возможно, и отказывать в получении кредита